# CONVOLUTIONAL NEURAL NETWORK: WHICH TOOL IT PROVIDES

Convolutional networks are a specialized kind of neural network for processing data that has a known grid-like topology. Examples include time-series data and image data. Convolution should be present in at least one of the layers.

![title](IMMAGINI/convolution.JPG)

the first argument is referred as input, the second argument (function w) as the kernel. The output is referred as feature map. In machine learning applications they refers to tensors that are multidimentional arrays. Many machine learning libraries call the cross-correlation as convolution. there are situations in which flipping he kernel would be important. Discrete convolution can be seen as a matrix moltiplication where the matrix should satisfy some constraints. Toeplitz matrix. sparse matrix (entries are mostly equal to zero)

###### WHAT THE CONVOLUTION IS

* SPARSE INTERACTIONS

CNN uses sparse interations, this is accomplished by making the kernel smaller that the input. In  this way fewer parameters are stored. if we limit the number of connection. m input and n outputs -> m x n: fully connected. k inputs n outputs sparsely connected ... kxn parameters.

* EQUIVARIANT REPRESENTATIONS

Equivariance to translation but not to rotation or scaling that require additional mechanism for handling these transformations. 

* PARAMETER SHARING

The same weight vector is used to compute the output not only in one postion. It means that the weight matrix contains elements that are revisited when computing the output of one layer. it follows an improvement from the point of view of memory requirements.

* INPUT OF VARIABLE SIZE

###### POOLING: an operation used by almost all CNN

THREE STAGES:

1. CONVOLUTION STAGES
2. DETECTOR STAGE: Applicaton of non linearity (e.g ReLU)
3. POOLING STAGE: modify the output of the layer further. A pooling function replaces the output of the net at a certain location with a summary statistic of the nearby outputs (e.g max pooling, average pooling). It introduces invariance to small changes.  pooling acts as a generalizer of the lower level data, and so, in a way, enables the network to move from high resolution data to lower resolution information. In other words, pooling coupled with convolutional filters attempts to detect objects within an image.

###### VARIANTS OF THE CONVOLUTION

###### CNN MORE EFFICIENT


# CNN IMPLEMENTATION

In [22]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch
# module for neural networks
import torch.nn as nn
# module that contains function such losses and activation functions
import torch.nn.functional as F

### Definition of the CNN class

###### DOUBTS
* how to choose dimension of kernel?
* How many convolutional layers?
* when it is necessary to use max pooling instead of average pooling?
* dimension of pooling?
* how Adam function precisely and how and if i can vary the learning rate
* when i prefer using an optimization algorithm instead of another one

At the end of a convolutional neural network it is possible to have a fully connected neural network. It is necessary to explain the reason of this choice. The fully connected layer can therefore be thought of as attaching a standard classifier onto the information-rich output of the network, to “interpret” the results and finally produce a classification result.  In order to attach this fully connected layer to the network, the dimensions of the output of the Convolutional Neural Network need to be flattened.


In [24]:
import torch
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

### Definition of Hiperparameters

In [38]:
#### Hyperparameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

DATA_PATH = 'Data'
MODEL_STORE_PATH = 'Models'

### Loading The dataset

The first thing to note above is the transforms.Compose() function. This function comes from the torchvision package. It allows the developer to setup various manipulations on the specified dataset. Numerous transforms can be chained together in a list using the Compose() function. In this case, first we specify a transform which converts the input data set to a PyTorch tensor. A PyTorch tensor is a specific data type used in PyTorch for all of the various data and weight operations within the network. In its essence though, it is simply a multi-dimensional matrix. In any case, PyTorch requires the data set to be transformed into a tensor so it can be consumed in the training and testing of the network.

The next argument in the Compose() list is a normalization transformation. Neural networks train better when the input data is normalized so that the data ranges from -1 to 1 or 0 to 1. To do this via the PyTorch Normalize transform, we need to supply the mean and standard deviation of the MNIST dataset, which in this case is 0.1307 and 0.3081 respectively. Note, that for each input channel a mean and standard deviation must be supplied – in the MNIST case, the input data is only single channeled, but for something like the CIFAR data set, which has 3 channels (one for each color in the RGB spectrum) you would need to provide a mean and standard deviation for each channel.

Next, the train_dataset and test_dataset objects need to be created. These will subsequently be passed to the data loader. In order to create these data sets from the MNIST data, we need to provide a few arguments. First, the root argument specifies the folder where the train.pt and test.pt data files exist. The train argument is a boolean which informs the data set to pickup either the train.pt data file or the test.pt data file. The next argument, transform, is where we supply any transform object that we’ve created to apply to the data set – here we supply the trans object which was created earlier. Finally, the download argument tells the MNIST data set function to download the data (if required) from an online source.


In [26]:
# transforms to apply to the data
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=True, transform=trans, download=True)
test_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=False, transform=trans)

In [27]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

The data loader object in PyTorch provides a number of features which are useful in consuming training data – the ability to shuffle the data easily, the ability to easily batch the data and finally, to make data consumption more efficient via the ability to load the data in parallel using multiprocessing. As can be observed, there are three simple arguments to supply – first the data set you wish to load, second the batch size you desire and finally whether you wish to randomly shuffle the data. A data loader can be used as an iterator – so to extract the data we can just use the standard Python iterators such as enumerate. This will be shown in practice later in this tutorial.

### Creation of the model

In [33]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

This is where the model definition takes place. The most straight-forward way of creating a neural network structure in PyTorch is by creating a class which inherits from the nn.Module super class within PyTorch. The nn.Module is a very useful PyTorch class which contains all you need to construct your typical deep learning networks. It also has handy functions such as ways to move variables and operations onto a GPU or back to a CPU, apply recursive functions across all the properties in the class (i.e. resetting all the weight variables), creates streamlined interfaces for training and so on. It is worth checking out all the methods available here.

The first step is to create some sequential layer objects within the class _init_ function. First, we create layer 1 (self.layer1) by creating a nn.Sequential object. This method allows us to create sequentially ordered layers in our network and is a handy way of creating a convolution + ReLU + pooling sequence. As can be observed, the first element in the sequential definition is the Conv2d nn.Module method – this method creates a set of convolutional filters. The first argument is the number of input channels – in this case, it is our single channel grayscale MNIST images, so the argument is 1. The second argument to Conv2d is the number of output channels – as shown in the model architecture diagram above, the first convolutional filter layer comprises of 32 channels, so this is the value of our second argument.

The kernel_size argument is the size of the convolutional filter – in this case we want 5 x 5 sized convolutional filters – so the argument is 5. If you wanted filters with different sized shapes in the x and y directions, you’d supply a tuple (x-size, y-size). Finally, we want to specify the padding argument. This takes a little bit more thought. The output size of any dimension from either a convolutional filtering or pooling operation can be calculated by the following equation:

![title](IMMAGINI/padding.JPG)

Where Win  is the width of the input, F is the filter size, P is the padding and S is the stride. The same formula applies to the height calculation, but seeing as our image and filtering are symmetrical the same formula applies to both. If we wish to keep our input and output dimensions the same, with a filter size of 5 and a stride of 1, it turns out from the above formula that we need a padding of 2. Therefore, the argument for padding in Conv2d is 2.

The next element in the sequence is a simple ReLU activation. The last element that is added in the sequential definition for self.layer1 is the max pooling operation. The first argument is the pooling size, which is 2 x 2 and hence the argument is 2. Second – we want to down-sample our data by reducing the effective image size by a factor of 2. To do this, using the formula above, we set the stride to 2 and the padding to zero. Therefore, the stride argument is equal to 2. The padding argument defaults to 0 if we don’t specify it – so that’s what is done in the code above. From these calculations, we now know that the output from self.layer1 will be 32 channels of 14 x 14 “images”.

Next, the second layer, self.layer2, is defined in the same way as the first layer. The only difference is that the input into the Conv2d function is now 32 channels, with an output of 64 channels. Using the same logic, and given the pooling down-sampling, the output from self.layer2 is 64 channels of 7 x 7 images.

Next, we specify a drop-out layer to avoid over-fitting in the model. Finally, two two fully connected layers are created. The first layer will be of size 7 x 7 x 64 nodes and will connect to the second layer of 1000 nodes. To create a fully connected layer in PyTorch, we use the nn.Linear method. The first argument to this method is the number of nodes in the layer, and the second argument is the number of nodes in the following layer.

Next, we specify a drop-out layer to avoid over-fitting in the model. Finally, two two fully connected layers are created. The first layer will be of size 7 x 7 x 64 nodes and will connect to the second layer of 1000 nodes. To create a fully connected layer in PyTorch, we use the nn.Linear method. The first argument to this method is the number of nodes in the layer, and the second argument is the number of nodes in the following layer.


With this _init_ definition, the layer definitions have now been created. The next step is to define how the data flows through these layers when performing the forward pass through the network:

It is important to call this function “forward” as this will override the base forward function in nn.Module and allow all the nn.Module functionality to work correctly. As can be observed, it takes an input argument x, which is the data that is to be passed through the model (i.e. a batch of data). We pass this data into the first layer (self.layer1) and return the output as “out”. This output is then fed into the following layer and so on. Note, after self.layer2, we apply a reshaping function to out, which flattens the data dimensions from 7 x 7 x 64 into 3164 x 1. Next, the dropout is applied followed by the two fully connected layers, with the final output being returned from the function.

### Training

Before we train the model, we have to first create an instance of our ConvNet class, and define our loss function and optimizer.
First, an instance of ConvNet() is created called “model”. Next, we define the loss operation that will be used to calculate the loss. In this case, we use PyTorch’s CrossEntropyLoss() function. You may have noticed that we haven’t yet defined a SoftMax activation for the final classification layer. This is because the CrossEntropyLoss function combines both a SoftMax activation and a cross entropy loss function in the same function – winning. 

In [34]:
model = ConvNet()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()

##### Adam Optimizer

Next, we define an Adam optimizer. The first argument passed to this function are the parameters we want the optimizer to train. This is made easy via the nn.Module class which ConvNet derives from – all we have to do is pass model.parameters() to the function and PyTorch keeps track of all the parameters within our model which are required to be trained. Finally, the learning rate is supplied. Adam is an optimization algorithm that can be used instead of the classical stochastic gradient descent procedure to update network weights iterative based in training data.

The attractive benefits of using Adam on non-convex optimization problems are:

* Straightforward to implement.
* Computationally efficient.
* Little memory requirements.
* Invariant to diagonal rescale of the gradients.
* Well suited for problems that are large in terms of data and/or parameters.
* Appropriate for non-stationary objectives.
* Appropriate for problems with very noisy/or sparse gradients.
* Hyper-parameters have intuitive interpretation and typically require little tuning.

CHECK WHAT IS ADAM

https://towardsdatascience.com/adam-latest-trends-in-deep-learning-optimization-6be9a291375c
https://www.youtube.com/watch?v=JXQT_vxqwIs

It combines the optimization coming from SGD with momentum and RMSprop. 
STEP:

1. definition of momentum using beta1
2. definition of variance using beta2
3. correction of momentum and variance 
4. use momentum and variance to evaluate weight update

eta: to be tuned
beta1 = 0.9 default value
beta2 = 0.999 default value
epsilon = 10^-8



In [35]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

##### Training Loop

In [36]:
# Train the model
total_step = len(train_loader)
loss_list = []
acc_list = []

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/5], Step [100/600], Loss: 0.1380, Accuracy: 96.00%
Epoch [1/5], Step [200/600], Loss: 0.2396, Accuracy: 95.00%
Epoch [1/5], Step [300/600], Loss: 0.1032, Accuracy: 97.00%
Epoch [1/5], Step [400/600], Loss: 0.1187, Accuracy: 94.00%
Epoch [1/5], Step [500/600], Loss: 0.0844, Accuracy: 98.00%
Epoch [1/5], Step [600/600], Loss: 0.1372, Accuracy: 96.00%
Epoch [2/5], Step [100/600], Loss: 0.0472, Accuracy: 99.00%
Epoch [2/5], Step [200/600], Loss: 0.1193, Accuracy: 97.00%
Epoch [2/5], Step [300/600], Loss: 0.1182, Accuracy: 97.00%
Epoch [2/5], Step [400/600], Loss: 0.0515, Accuracy: 97.00%
Epoch [2/5], Step [500/600], Loss: 0.0685, Accuracy: 97.00%
Epoch [2/5], Step [600/600], Loss: 0.0665, Accuracy: 96.00%
Epoch [3/5], Step [100/600], Loss: 0.1208, Accuracy: 98.00%
Epoch [3/5], Step [200/600], Loss: 0.1568, Accuracy: 96.00%
Epoch [3/5], Step [300/600], Loss: 0.1152, Accuracy: 97.00%
Epoch [3/5], Step [400/600], Loss: 0.1061, Accuracy: 97.00%
Epoch [3/5], Step [500/600], Loss: 0.282

The most important parts to start with are the two loops – first, the number of epochs is looped over, and within this loop, we iterate over train_loader using enumerate. Within this inner loop, first the outputs of the forward pass through the model are calculated by passing images (which is a batch of normalized MNIST images from train_loader) to it. Note, we don’t have to call model.forward(images) as nn.Module knows that forward needs to be called when it executes model(images).

The next step is to pass the model outputs and the true image labels to our CrossEntropyLoss function, defined as criterion. The loss is appended to a list that will be used later to plot the progress of the training. The next step is to perform back-propagation and an optimized training step. First, the gradients have to be zeroed, which can be done easily by calling zero_grad() on the optimizer. Next, we call .backward() on the loss variable to perform the back-propagation. Finally, now that the gradients have been calculated in the back-propagation, we simply call optimizer.step() to perform the Adam optimizer training step. PyTorch makes training the model very easy and intuitive.

The next set of steps involves keeping track of the accuracy on the training set. The predictions of the model can be determined by using the torch.max() function, which returns the index of the maximum value in a tensor. The first argument to this function is the tensor to be examined, and the second argument is the axis over which to determine the index of the maximum. The output tensor from the model will be of size (batch_size, 10). To determine the model prediction, for each sample in the batch we need to find the maximum value over the 10 output nodes. Each of these will correspond to one of the hand written digits (i.e. output 2 will correspond to digit “2” and so on). The output node with the highest value will be the prediction of the model. Therefore, we need to set the second argument of the torch.max() function to 1 – this points the max function to examine the output node axis (axis=0 corresponds to the batch_size dimension).

This returns a list of prediction integers from the model – the next line compares the predictions with the true labels (predicted == labels) and sums them to determine how many correct predictions there are. Note the output of sum() is still a tensor, so to access it’s value you need to call .item(). We divide the number of correct predictions by the batch_size (equivalent to labels.size(0)) to obtain the accuracy. Finally, during training, after every 100 iterations of the inner loop the progress is printed.

#### Testing the model

In [39]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct / total) * 100))

# Save the model and plot
torch.save(model.state_dict(), MODEL_STORE_PATH + 'conv_net_model.ckpt')

Test Accuracy of the model on the 10000 test images: 98.97 %


As a first step, we set the model to evaluation mode by running model.eval(). This is a handy function which disables any drop-out or batch normalization layers in your model, which will befuddle your model evaluation / testing. The torch.no_grad() statement disables the autograd functionality in the model (see here for more details) as it is not needing in model testing / evaluation, and this will act to speed up the computations. The rest is the same as the accuracy calculations during training, except that in this case, the code iterates through the test_loader.

Finally, the result is output to the console, and the model is saved using the torch.save() function.

### Plot

In [40]:
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.models import LinearAxis, Range1d
import numpy as np


In [41]:
p = figure(y_axis_label='Loss', width=850, y_range=(0, 1), title='PyTorch ConvNet results')
p.extra_y_ranges = {'Accuracy': Range1d(start=0, end=100)}
p.add_layout(LinearAxis(y_range_name='Accuracy', axis_label='Accuracy (%)'), 'right')
p.line(np.arange(len(loss_list)), loss_list)
p.line(np.arange(len(loss_list)), np.array(acc_list) * 100, y_range_name='Accuracy', color='red')
show(p)